## Equipo ...CriptoBroker...
#### Asignatura: TEL252-S2-2020
#### Alumnos: Christian Fuentes 201530024-0 , Mario Araya 2015 201630003-1 , Sebastián Araneda 201530022-4 , Victor Cortés 201604140-0


### El producto consisten 4 partes fundamentales:
     1.-  Frontend
     2.-  DB
     3.-  Api
     4.-  Sensor

## 1.- Frontend
En la parte del frontend lo que tenemos es una vista la cual va a mostrar la temperatura de los sensores de forma gráfica, 
donde la gráfica es de temperatura versus tiempo, tomando la temperatura de un sensor y gráficándolo en un determinado intervalo de tiempo, ademas del identificador unido del sensor

## 2.- Base de datos
La base de datos se encuanta en un servidor alojado en DigitalOcean, es una base datos mysql donde tenemos la tabla "sensores" que contiene las variables "id_sensor" , "timestamp" y "lectura". Cabe destacar que el unico que tiene conexion directa con la base de datos es la API por el puerto 9600.

## 3.- API
Para nuestro caso de la api rest, esta vez usamos express con con nodejs para recibir las solicitudes de get y post que van a llegar ya sea para pedir la temperatura de la base de datos o para ingresarla cuando el sensor entregue los datos para ser guardados en la base de datos.

## 4.- Sensor
Para el caso de nuestro sensor, lo que tenemos es un archivo sensor.py que simula un sensor, mandando la temperatura  encriptada usando el algoritmo de "AES", para posteriormente ser enviada en una trama http que contiene un json de la forma 
```
{
         "id_sensor": id_sensor,
         'lectura': data.decode("utf-8")
}
```
donde el "id_sensor" corresponde a un valor que diferencia un sensor de otro en caso de que tengamos más de uno, y la lectura corresponde al valor de temperatura en nuestro caso que nos entrega el sensor. 

## Proyección:
-Para el futuro desarrollo del proyecto lo que queremos hacer es dejar la base de datos conectada de forma remota, con el objetivo de poder tenerla en cualquier momento y no tener un manejo de forma local, sino uno más cercano a lo que es la  realidad, además vamos a implementar una capa más de seguridad que corresponderá  al firewall, cosa de complicar la forma en la que eve va a tratar de romper u obtener algún dato que sea de caracter sensible, otra cosa a implementar viene siendo el micro kernel que va a alojar a nuestro sensor y por último mejorar nuestra forma de encriptar nuestros datos de  temperatura, esto último es algo que sigue en conversación, ya que se quiere elegir una forma tal que no sea facil de  romper ni tampoco que consuma tantos recursos, por tanto que sea eficiente.